# Notebook 1 — Data Loading & Preparation

This notebook merges the city-level Airbnb CSV files, performs basic cleaning/standardization, creates a few engineered features, and saves a prepared dataset for EDA, hypothesis testing, and ML.


In [20]:
import os
import glob
import zipfile
import re
import ast
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)


## Load and Extract Data

Locate the uploaded ZIP file, extract its contents, and identify all city-level CSV files.


In [21]:
zip_candidates = [
    "/content/airbnb_data.zip",           # if you renamed it
    "/content/airbnb_data (1).zip",
    "/content/airbnb_data (2).zip",
    "/content/airbnb_data (3).zip",
]

zip_path = None
for z in zip_candidates:
    if os.path.exists(z):
        zip_path = z
        break

if zip_path is None:
    # fallback: any zip in /content
    zips = glob.glob("/content/*.zip")
    if len(zips) > 0:
        zip_path = zips[0]

if zip_path is None:
    raise FileNotFoundError("Zip file not found in /content. Upload the dataset zip first.")

extract_dir = "/content/airbnb_data_extracted"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zf:
    zf.extractall(extract_dir)

csv_files = glob.glob(os.path.join(extract_dir, "**/*.csv"), recursive=True)
if len(csv_files) == 0:
    raise FileNotFoundError("No CSV files found inside extracted zip.")

print(f"Using zip: {zip_path}")
print(f"Extracted to: {extract_dir}")
print(f"Found CSV files: {len(csv_files)}")
print("Example:", csv_files[:5])


Using zip: /content/airbnb_data.zip
Extracted to: /content/airbnb_data_extracted
Found CSV files: 9
Example: ['/content/airbnb_data_extracted/TorontoData.csv', '/content/airbnb_data_extracted/SanFransiscoData.csv', '/content/airbnb_data_extracted/LondonData.csv', '/content/airbnb_data_extracted/NYCData.csv', '/content/airbnb_data_extracted/TokyoData.csv']


## Merge City-Level Files

Each city CSV is loaded separately. A `city_file` column is added to preserve the
source city, and all files are merged into a single DataFrame.


In [22]:
dfs = []
for f in csv_files:
    temp = pd.read_csv(f)
    # keep which file/city it came from
    temp["city_file"] = os.path.splitext(os.path.basename(f))[0]
    dfs.append(temp)

raw = pd.concat(dfs, ignore_index=True)
print("Merged shape:", raw.shape)
raw.head()


Merged shape: (145825, 58)


,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Currency Native,Number of Reviews,Bedrooms,Bathrooms,Max Guests,Airbnb Superhost,Cancellation Policy,Cleaning Fee (USD),Cleaning Fee (Native),Extra People Fee (USD),Extra People Fee(Native),Check-in Time,Checkout Time,Minimum Stay,Latitude,Longitude,Exact Location,Overall Rating,Airbnb Communication Rating,Airbnb Accuracy Rating,Airbnb Cleanliness Rating,Airbnb Checkin Rating,Airbnb Location Rating,Airbnb Value Rating,Amenities,picture_url,License,Airbnb Property ID,Airbnb Host ID,Host Listing Count,guest_controls,instant_bookable,Pets Allowed,Listing URL,Instantbook Enabled,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native),city_file,Zipcode,Last Host Count Updated Date,Neighbourhood,Metropolitan Statistical Area
0,Waterfront Cozy Escape,Private room in rental unit,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,79,2,1,4,t,flexible,80.0,107.2,7.98889,10.7,After 3:00 PM,11:00 AM,3,43.587961,-79.53622,f,4.8,10.0,10.0,10.0,10.0,10.0,10.0,"['Free parking on premises', 'Wifi', 'TV', 'Ha...",https://a0.muscache.com/im/pictures/miso/Hosti...,STR-2106-HVBPHL,35315385,199794462,1.0,"{""allows_children"": false, ""allows_infants"": f...",0.0,False,http://airbnb.com/rooms/35315385,False,224,141,162,87,26,23,100,134.0,21568,28901.1,TorontoData,NaN,NaN,NaN,NaN
1,Live by the Lake Ontario-Entire Apartment,Entire condo,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,19,2,1,4,f,moderate,45.0,60.3,0.00000,0.0,3:00 PM - 10:00 PM,11:00 AM,28,43.589160,-79.53253,f,4.8,10.0,10.0,10.0,10.0,10.0,10.0,"['Free parking on premises', 'Wifi', 'Kitchen'...",https://a0.muscache.com/im/pictures/miso/Hosti...,STR-2010-GKWKPG,41267906,309589086,1.0,"{""allows_children"": true, ""allows_infants"": tr...",0.0,False,http://airbnb.com/rooms/41267906,False,32,333,19,71,2,-1,102,136.7,2407,3225.4,TorontoData,NaN,NaN,NaN,NaN
2,Home away from home.,Private room in rental unit,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,5,1,1,2,f,flexible,7.0,9.4,0.00000,0.0,After 5:00 PM,10:00 AM,28,43.588320,-79.53094,f,5.0,10.0,10.0,10.0,10.0,10.0,9.0,"['Free parking on premises', 'Elevator', 'Wifi...",https://a0.muscache.com/im/pictures/miso/Hosti...,NaN,41213227,39378580,1.0,"{""allows_children"": false, ""allows_infants"": f...",1.0,False,http://airbnb.com/rooms/41213227,True,173,192,73,51,3,0,74,99.2,6550,8777.0,TorontoData,NaN,NaN,NaN,NaN
3,"❤️Beautiful HOUSE!, near everything! LAKE+WIFI...",Entire home,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,50,1,1,2,t,strict_14_with_grace_period,118.0,158.1,0.00000,0.0,After 3:00 PM,11:00 AM,28,43.590000,-79.52848,f,4.9,10.0,10.0,10.0,10.0,10.0,10.0,"['Free parking on premises', 'Wifi', 'Kitchen'...",https://a0.muscache.com/im/pictures/944e5614-5...,STR-2205-HCGPHJ,48123819,386402738,6.0,"{""allows_children"": true, ""allows_infants"": tr...",0.0,False,http://airbnb.com/rooms/48123819,False,78,287,33,51,5,2,65,87.1,3176,4255.8,TorontoData,NaN,NaN,NaN,NaN
4,💕BEAUTIFUL BSMT STUDIO By LAKE! + wifi PRKN & ...,Entire home,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,46,1,1,2,t,strict_14_with_grace_period,118.0,158.1,15.73330,21.1,After 3:00 PM,NaN,28,43.591560,-79.52895,f,5.0,10.0,10.0,10.0,10.0,10.0,10.0,"['Free parking on premises', 'Breakfast', 'Wif...",https://a0.muscache.com/im/pictures/85005a96-c...,STR-2205-HCGPHJ,47940902,386402738,6.0,"{""allows_children"": false, ""allows_infants"": f...",0.0,False,http://airbnb.com/rooms/47940902,False,271,94,153,68,21,18,84,112.6,17958,24063.7,TorontoData,NaN,NaN,NaN,NaN


## Data Cleaning and Standardization

Standardize column names, parse date columns, fix data types, normalize boolean values,
and remove duplicate records.


In [23]:
df = raw.copy()

def to_snake(s):
    s = s.strip().lower()
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\s+", "_", s)
    return s

df.columns = [to_snake(c) for c in df.columns]

# dates
for col in ["created_date", "last_scraped_date"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# city file clean label
df["city_file_clean"] = (
    df["city_file"].astype(str)
      .str.replace("data", "", case=False, regex=False)
      .str.replace("dataset", "", case=False, regex=False)
)

# make booleans consistent
bool_cols = ["airbnb_superhost", "instant_bookable", "pets_allowed"]
for c in bool_cols:
    if c in df.columns:
        df[c] = df[c].replace({"t": True, "f": False, "True": True, "False": False})

# bedrooms sometimes object -> numeric
if "bedrooms" in df.columns:
    df["bedrooms"] = pd.to_numeric(df["bedrooms"], errors="coerce")

# drop duplicates
df = df.drop_duplicates()

df.shape


/tmp/ipython-input-1401729896.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({"t": True, "f": False, "True": True, "False": False})


(145825, 59)

## Feature Engineering

Create additional features to enrich the dataset, including the number of amenities
per listing.


In [24]:
# amenities_count
if "amenities" in df.columns:
    def count_amenities(x):
        if pd.isna(x):
            return np.nan
        # sometimes it's "['Wifi', 'Kitchen']" as string
        if isinstance(x, str):
            try:
                x = ast.literal_eval(x)
            except:
                return np.nan
        if isinstance(x, list):
            return len(x)
        return np.nan

    df["amenities_count"] = df["amenities"].apply(count_amenities)

# target = annual_revenue_ltm_usd (regression goal later)
target = "annual_revenue_ltm_usd"
if target not in df.columns:
    raise KeyError("annual_revenue_ltm_usd is missing. Check dataset columns.")

# select a clean subset
keep_cols = [
    "listing_title","property_type","listing_type","amenities",
    "country","state","city","city_file_clean",
    "bedrooms","bathrooms","max_guests","minimum_stay","number_of_reviews",
    "overall_rating","airbnb_value_rating","airbnb_location_rating",
    "airbnb_cleanliness_rating","airbnb_accuracy_rating","airbnb_communication_rating",
    "cleaning_fee_usd","extra_people_fee_usd","average_daily_rate_usd",
    "occupancy_rate_ltm","annual_revenue_ltm_usd",
    "count_available_days_ltm","count_blocked_days_ltm","count_reservation_days_ltm",
    "number_of_bookings_ltm",
    "airbnb_superhost","instant_bookable","pets_allowed",
    "latitude","longitude","created_date","last_scraped_date",
    "amenities_count"
]

keep_cols = [c for c in keep_cols if c in df.columns]
df_small = df[keep_cols].copy()

print("Selected columns:", len(keep_cols))
print("Shape:", df_small.shape)
df_small.head(3)


Selected columns: 36
Shape: (145825, 36)


,listing_title,property_type,listing_type,amenities,country,state,city,city_file_clean,bedrooms,bathrooms,max_guests,minimum_stay,number_of_reviews,overall_rating,airbnb_value_rating,airbnb_location_rating,airbnb_cleanliness_rating,airbnb_accuracy_rating,airbnb_communication_rating,cleaning_fee_usd,extra_people_fee_usd,average_daily_rate_usd,occupancy_rate_ltm,annual_revenue_ltm_usd,count_available_days_ltm,count_blocked_days_ltm,count_reservation_days_ltm,number_of_bookings_ltm,airbnb_superhost,instant_bookable,pets_allowed,latitude,longitude,created_date,last_scraped_date,amenities_count
0,Waterfront Cozy Escape,Private room in rental unit,private_room,"['Free parking on premises', 'Wifi', 'TV', 'Ha...",CA,Ontario,Toronto,Toronto,2.0,1,4,3,79,4.8,10.0,10.0,10.0,10.0,10.0,80.0,7.98889,100,87,21568,224,141,162,26,True,0.0,False,43.587961,-79.53622,2023-07-31,2024-01-10,29
1,Live by the Lake Ontario-Entire Apartment,Entire condo,entire_home,"['Free parking on premises', 'Wifi', 'Kitchen'...",CA,Ontario,Toronto,Toronto,2.0,1,4,28,19,4.8,10.0,10.0,10.0,10.0,10.0,45.0,0.00000,102,71,2407,32,333,19,2,False,0.0,False,43.589160,-79.53253,2023-07-31,2024-01-10,31
2,Home away from home.,Private room in rental unit,private_room,"['Free parking on premises', 'Elevator', 'Wifi...",CA,Ontario,Toronto,Toronto,1.0,1,2,28,5,5.0,9.0,10.0,10.0,10.0,10.0,7.0,0.00000,74,51,6550,173,192,73,3,False,1.0,False,43.588320,-79.53094,2023-07-31,2024-01-10,56


## Final Dataset Preparation

Select analysis features, validate the target variable (annual revenue), and save the
prepared dataset for later analysis.


In [26]:
# drop missing target
before = len(df_small)
df_small = df_small.dropna(subset=["annual_revenue_ltm_usd"])
print("Dropped rows missing target:", before - len(df_small))

# basic sanity: no negative money values
money_cols = ["annual_revenue_ltm_usd", "average_daily_rate_usd", "cleaning_fee_usd", "extra_people_fee_usd"]
for c in money_cols:
    if c in df_small.columns:
        df_small = df_small[df_small[c].isna() | (df_small[c] >= 0)]

out_dir = "/content/data/processed"
os.makedirs(out_dir, exist_ok=True)
prepared_path = os.path.join(out_dir, "airbnb_prepared.csv")
df_small.to_csv(prepared_path, index=False)

print("Saved:", prepared_path)
print("Final shape:", df_small.shape)
df_small.sample(5, random_state=42)


Dropped rows missing target: 0
Saved: /content/data/processed/airbnb_prepared.csv
Final shape: (145825, 36)


,listing_title,property_type,listing_type,amenities,country,state,city,city_file_clean,bedrooms,bathrooms,max_guests,minimum_stay,number_of_reviews,overall_rating,airbnb_value_rating,airbnb_location_rating,airbnb_cleanliness_rating,airbnb_accuracy_rating,airbnb_communication_rating,cleaning_fee_usd,extra_people_fee_usd,average_daily_rate_usd,occupancy_rate_ltm,annual_revenue_ltm_usd,count_available_days_ltm,count_blocked_days_ltm,count_reservation_days_ltm,number_of_bookings_ltm,airbnb_superhost,instant_bookable,pets_allowed,latitude,longitude,created_date,last_scraped_date,amenities_count
13770,Camberwell flat,Entire rental unit,entire_home,"['Wifi', 'Kitchen', 'Dedicated workspace', 'TV...",GB,UK,Greater London,London,1.0,1,2,3,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,134,90,482,4,361,3,2,False,NaN,False,51.46700,-0.098000,2023-05-22,2023-07-05,33
62403,A cozy bedroom.,Private room in home,private_room,"['Free parking on premises', 'Iron', 'Dedicate...",US,New Jersey,Fair Lawn,NYC,1.0,1,1,1,5,5.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,49,89,3925,90,275,67,9,True,NaN,False,40.92245,-74.120990,2023-07-31,2024-01-10,8
78984,- The Bricks Putnam - Pre-war updated house,Entire townhouse,entire_home,"['Kitchen', 'Wifi', 'Washer', 'Dryer', 'Iron',...",US,New York,Brooklyn,NYC,2.0,2,7,2,49,4.9,10.0,9.0,10.0,10.0,10.0,0.0,0.0,256,73,65595,323,42,187,42,True,NaN,False,40.69100,-73.915000,2023-07-31,2023-12-13,37
23617,LARGE ROOM SLEEPS 4 IN LONDON,Private room in rental unit,private_room,"['Wifi', 'Kitchen', 'TV', 'Hangers', 'Heating'...",GB,UK,London,London,1.0,1,4,2,24,4.4,10.0,9.0,10.0,10.0,10.0,0.0,0.0,47,30,56,4,361,1,0,False,0.0,False,51.43300,0.079370,2022-10-08,2022-11-16,22
125219,Cozy Apt. In Miami/Wynwood Centrally Located,Entire rental unit,entire_home,"['Free parking on premises', 'Kitchen', 'Wifi'...",US,Florida,Miami,Miami,2.0,1,6,2,13,5.0,10.0,9.0,10.0,10.0,10.0,NaN,NaN,101,89,11416,127,238,94,13,True,NaN,False,25.77755,-80.212274,2023-07-31,2023-12-27,31
